## Trying to build a model with transfer learning and word embeddings

Following: https://machinelearningmastery.com/how-to-develop-a-word-level-neural-language-model-in-keras/

In [27]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [1]:
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, LSTM, Flatten, Embedding, TimeDistributed, Activation
from keras.optimizers import RMSprop
from collections import Counter
from datetime import datetime
import keras
import keras.callbacks
from keras.callbacks import TensorBoard
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical
from keras.preprocessing.text import one_hot
from pickle import dump

import numpy as np 
import string
import random

In [12]:
# documents 
with open('Reddit_Blonde_short.txt', 'r') as file: 
#with open("One_liners.txt", 'r') as file:
    doc = file.read()

In [14]:
print(doc[:10])

A middle a


In [15]:
# turn a doc into clean tokens
def clean_doc(doc):
    # replace '--' with a space ' '
    doc = doc.replace('--', ' ')
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # make lower case
    tokens = [word.lower() for word in tokens]
    return tokens

In [16]:
# clean document
tokens = clean_doc(doc)
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

['a', 'middle', 'aged', 'frumpy', 'married', 'couple', 'return', 'to', 'a', 'mercedes', 'dealership', 'where', 'the', 'salesman', 'has', 'just', 'sold', 'the', 'car', 'they', 'were', 'interested', 'in', 'to', 'a', 'beautiful', 'leggy', 'busty', 'blonde', 'i', 'thought', 'you', 'said', 'you', 'would', 'hold', 'that', 'car', 'till', 'we', 'raised', 'the', 'asking', 'price', 'said', 'the', 'man', 'i', 'just', 'heard', 'you', 'close', 'the', 'deal', 'for', 'to', 'the', 'lovely', 'young', 'lady', 'there', 'you', 'insisted', 'there', 'could', 'be', 'no', 'discount', 'on', 'the', 'model', 'well', 'she', 'had', 'the', 'ready', 'cash', 'and', 'just', 'look', 'at', 'her', 'how', 'could', 'i', 'resist', 'replied', 'the', 'grinning', 'salesman', 'just', 'then', 'the', 'young', 'woman', 'approached', 'the', 'middle', 'aged', 'couple', 'and', 'gave', 'them', 'the', 'keys', 'there', 'you', 'go', 'she', 'said', 'i', 'told', 'you', 'i', 'would', 'get', 'the', 'prick', 'to', 'reduce', 'it', 'see', 'you'

In [44]:
length = 50 + 1
sequences = list()
for i in range(length, len(tokens)):
    # select sequence of tokens
    seq = tokens[i-length:i]
    # convert into a line
    line = ' '.join(seq)
    # store
    sequences.append(line)
#sequences = the text divided into len 50 
print('Total Sequences: %d' % len(sequences))
print()
print(sequences[0])

Total Sequences: 16856

a middle aged frumpy married couple return to a mercedes dealership where the salesman has just sold the car they were interested in to a beautiful leggy busty blonde i thought you said you would hold that car till we raised the asking price said the man i just heard you


In [45]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sequences)
sequences = tokenizer.texts_to_sequences(sequences)

vocab_size = len(tokenizer.word_index) + 1
print("Vocab size:", vocab_size)

print()
print(sequences[0])

Vocab size: 2387

[2, 519, 1210, 2386, 1209, 518, 430, 4, 2, 2385, 2384, 276, 1, 160, 159, 58, 2382, 1, 69, 22, 51, 1208, 10, 4, 2, 192, 2377, 2375, 5, 9, 311, 8, 48, 8, 82, 1207, 17, 69, 2374, 39, 1206, 1, 837, 2372, 48, 1, 59, 9, 58, 275, 8]


In [46]:
max_length = 50

In [47]:
# separate into input and output
sequences = np.array(encoded_jokes)
X = sequences[:,:-1]
y = sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [12]:
# Embeddings 

In [48]:
# TAKES A LONG TIME TO LOAD
# download pre-made embeddings 
embeddings_index = dict()
f = open('glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [49]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [53]:
# define embedding 
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_length, trainable=False)

In [78]:
# define model
model = Sequential()
#model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(e)
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 100)           238700    
_________________________________________________________________
lstm_8 (LSTM)                (None, 50, 100)           80400     
_________________________________________________________________
lstm_9 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_8 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_9 (Dense)              (None, 2387)              241087    
Total params: 650,687
Trainable params: 411,987
Non-trainable params: 238,700
_________________________________________________________________
None


In [83]:
# fit the model
model.fit(X, y, batch_size=128, epochs=100)

Epoch 1/100
132/132 [==============================] - 11s 82ms/step - loss: 0.0027 - accuracy: 0.1018
Epoch 2/100
132/132 [==============================] - 11s 86ms/step - loss: 0.0027 - accuracy: 0.1029
Epoch 3/100
132/132 [==============================] - 11s 86ms/step - loss: 0.0026 - accuracy: 0.1057
Epoch 4/100
132/132 [==============================] - 12s 91ms/step - loss: 0.0026 - accuracy: 0.1079
Epoch 5/100
132/132 [==============================] - 12s 92ms/step - loss: 0.0026 - accuracy: 0.1114
Epoch 6/100
132/132 [==============================] - 12s 93ms/step - loss: 0.0026 - accuracy: 0.1166
Epoch 7/100
132/132 [==============================] - 15s 115ms/step - loss: 0.0025 - accuracy: 0.1191
Epoch 8/100
132/132 [==============================] - 16s 124ms/step - loss: 0.0025 - accuracy: 0.1249
Epoch 9/100
132/132 [==============================] - 18s 134ms/step - loss: 0.0025 - accuracy: 0.1318
Epoch 10/100
132/132 [==============================] - 18s 137ms/step

132/132 [==============================] - 18s 136ms/step - loss: 3.8432e-04 - accuracy: 0.8622
Epoch 79/100
132/132 [==============================] - 18s 135ms/step - loss: 3.7916e-04 - accuracy: 0.8647
Epoch 80/100
132/132 [==============================] - 18s 134ms/step - loss: 3.6065e-04 - accuracy: 0.8722
Epoch 81/100
132/132 [==============================] - 17s 132ms/step - loss: 3.4929e-04 - accuracy: 0.8748
Epoch 82/100
132/132 [==============================] - 19s 140ms/step - loss: 3.4238e-04 - accuracy: 0.8803
Epoch 83/100
132/132 [==============================] - 17s 127ms/step - loss: 3.2839e-04 - accuracy: 0.8841
Epoch 84/100
132/132 [==============================] - 18s 134ms/step - loss: 3.1987e-04 - accuracy: 0.8919
Epoch 85/100
132/132 [==============================] - 19s 141ms/step - loss: 3.0018e-04 - accuracy: 0.8976
Epoch 86/100
132/132 [==============================] - 18s 134ms/step - loss: 2.8578e-04 - accuracy: 0.9054
Epoch 87/100
132/132 [==========

In [81]:
# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
    result = list()
    result.append(seed_text)
    in_text = seed_text
    # generate a fixed number of words
    saveprediction = [[0]]
    for _ in range(n_words):
        # encode the text as integer
        encoded = tokenizer.texts_to_sequences([in_text])[0] # model is predicting what comes after this
        #print(len(encoded), in_text)
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        
        # predict probabilities for each word
        predictions = model.predict(encoded, verbose=0)[0]#[1:]
        #print(len(predictions))
        yhat = np.argmax(predictions)
        #print(np.argmax(predictions[0]))
        #yhat = random.randint(0, len(predictions))
        #print(yhat)
        
#         yhat = model.predict(encoded, verbose=0)[0][1:]
#         if np.array_equal(yhat, saveprediction):
#             print("Nothing new")
#         saveprediction = yhat
#         testList = list(yhat)
#         indices = []
#         for i in range(5):
#             temp = np.argmax(testList)
#             indices.append(temp)
#             testList.pop(temp)
#         yhat = np.argmax(yhat)
#         yhat = random.choice(indices)

        
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            #print("check:", word, index, yhat)
            if index == yhat:
                #print("out:", word, index)
                out_word = word
                break
        # append to input
        in_text += ' ' + out_word
        result.append(out_word)
    return ' '.join(result)

In [82]:
seed_text = "a blonde arrives at work one morning"
# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 100)
print(generated)

a blonde arrives at work one morning blondes conceive consideration blondes conceive consideration consideration blondes conceive consideration blondes conceive consideration consideration blondes conceive consideration consideration blondes conceive consideration consideration blondes conceive consideration consideration blondes conceive consideration consideration blondes conceive consideration consideration blondes conceive consideration consideration blondes conceive consideration consideration blondes conceive consideration consideration blondes conceive consideration consideration blondes conceive consideration consideration blondes conceive consideration consideration blondes conceive consideration consideration blondes conceive consideration consideration blondes conceive consideration consideration blondes conceive consideration consideration blondes conceive consideration consideration blondes conceive consideration consideration blondes conceive consideration consideration b